In [12]:
import re
import datetime
import json
import uuid


class Bank:
  def __init__(self):
    self.transactions={}
    self.bank_details={}
    self.user_account_data={}
    self.accounts={}

  #HER YAPILAN İŞLEMİN KALICI OLARAK KAYDEDİLMESİ VE DİĞER YAPILACAK İŞLEMLERİNDE KAYITLARA GÖRE DEVAM ETMESİ GEREKİYORç
  #İŞLEMLERİ LOGLUYORUZ:

  def log_transaction(self,transaction_type,amount,acc_no=None,sender_acc_no=None,reciever_acc_no=None):
    date_str=str(datetime.datetime.now())
    self.transactions[date_str]={
        "transaction_type":transaction_type,
        "amount":amount,
        "acc_no":acc_no,
        "sender_acc_no":sender_acc_no,
        "reciever_acc_no":reciever_acc_no
    }


  def deposite_Amount(self,accounts,transactions):
    print("""
                                  DEPOSİTE AMOUNT
    """)
    self.acc_no=input("Account Number: ")
    if len(self.acc_no)!=14:
      return f"Account Number should be 14 digits long,please re-check"
    if re.findall("[a-z]",self.acc_no):
      return f"Account Number should have numbers"
    else:
      if self.acc_no in accounts.keys():
        print(f"""
        User Details {accounts[self.acc_no]}
        """)
        self.amount=int(input("Deposite Amount: "))
        if self.amount<=0:
          return f"Amount should be more than 0"
        else:
          accounts[self.acc_no]["balance"]+=self.amount
          self.log_transaction("Deposit",self.amount,acc_no=self.acc_no)

          transactions[str(datetime.date.today())]={}
          transactions[str(datetime.date.today())]["statement"]=f"{self.amount} credited to {self.acc_no}"



          with open("user_account.json",'w') as customer_file:
            json.dump(accounts,customer_file,indent=4)

          with open("transac.json",'a') as transac_file:
            json.dump(transactions,transac_file,indent=4)
      else:
        return f"{self.acc_no} does not exist in our database"


  def withdraw_amount(self,accounts,transactions):
    print("""
                              WİTHDRAW AMOUNT
    """)
    self.acc_no=input("Account Number: ")
    if len(self.acc_no)!=14:
      return f"Account Number should be 14 digits long,please re-check"
    if re.findall("[a-z]",self.acc_no):
      return f"Account Number should have numbers"
    else:
      if self.acc_no in accounts.keys():
        print(f"""
        User Details {accounts[self.acc_no]}
        """)
        self.amount=int(input("Withdraw Amount: "))
        if self.amount<=0:
          return f"Amount should be more than 0"
        else:
          accounts[self.acc_no]["balance"]-=self.amount
          self.log_transaction("Withdrawal", self.amount, acc_no=self.acc_no)

          transactions[str(datetime.date.today())]={}
          transactions[str(datetime.date.today())]["statement"]=f"{self.amount} debited to {self.acc_no}"



          with open("user_account.json",'w') as customer_file:
            json.dump(accounts,customer_file,indent=4)

          with open("transac.json",'a') as transac_file:
            json.dump(transactions,transac_file,indent=4)
      else:
        return f"{self.acc_no} does not exist in our database"



#para transfer işleminde bir gönderen bir alıcı var,gönderenin hesapno,alıcı hesapno
#iki hesap numarasının da geçerlilik kontrolü(14 digitli mi)
#hesap numaralı hesaplar listesinde var mı yok mu kontrol
#miktar 0 dan büyük olacak gönderenin göndereceği miktar bakiyesinden büyük ise yetersiz bakiye error geri dönüş mesjı
#transfer işlemi sayısal gerçekleştir gönderenin bakiyesinden miktarı azalt
#alıcının bakiyesinde miktarı ekle arttır
#işlemi kaydet ve başarılı geri dönüt paylaş


  def transfer_money(self, accounts, transactions):
    print("""
                              TRANSFER AMOUNT
    """)
    self.sender_acc_no = input("Sender Account Number: ")
    self.reciever_acc_no = input("Receiver Account Number: ")

    if len(self.sender_acc_no) != 14 or len(self.reciever_acc_no) != 14:
        return f"Both Account Numbers should be 14 digits long, please re-check."

    if re.findall("[a-z]", self.sender_acc_no) or re.findall("[a-z]", self.reciever_acc_no):
        return f"Both Account Numbers should have only numbers."

    if self.sender_acc_no not in accounts.keys():
        return f"Sender account number {self.sender_acc_no} does not exist in our database."

    if self.reciever_acc_no not in accounts.keys():
        return f"Receiver account number {self.reciever_acc_no} does not exist in our database."

    self.amount = int(input("Transfer Amount: "))
    if self.amount <= 0:
        return f"Amount should be more than 0."

    if accounts[self.sender_acc_no]["balance"] < self.amount:
        return f"Insufficient balance."


    accounts[self.sender_acc_no]["balance"] -= self.amount
    accounts[self.reciever_acc_no]["balance"] += self.amount
    self.log_transaction("Transfer",  self.amount, sender_acc_no=self.sender_acc_no, reciever_acc_no=self.reciever_acc_no)


    transactions[str(datetime.date.today())] = {}
    transactions[str(datetime.date.today())]["statement"] = f"{self.sender_acc_no}  is {self.amount} TRANSFERRED to {self.reciever_acc_no}"


    with open("user_account.json", 'w') as customer_file:
        json.dump(accounts, customer_file, indent=4)

    with open("transac.json", 'a') as transac_file:
        json.dump(transactions, transac_file, indent=4)

    return f"{self.amount} transferred successfully from {self.sender_acc_no} to {self.reciever_acc_no}."



  def generate_unique_account_number(self):
    unique_number=str(uuid.uuid4().int)[:14]
    return unique_number

  def load_accounts(self,customer_file="user_account.json"):
    with open(customer_file,'r') as customer_file:
      accounts=json.load(customer_file)
    return accounts

  def save_accounts(self,accounts,customer_file="user_account.json"):
    with open(customer_file,'w') as customer_file:
      json.dump(accounts,customer_file,indent=4)

  def create_account(self,accounts):
    self.acc_no=self.generate_unique_account_number()
    while self.acc_no in accounts:
      self.acc_no=self.generate_unique_account_number()

    name=input("Enter account holder's name: ")
    age=int(input("Enter account holder's age: "))
    gender=input("Enter account holder's gender: ")
    initial_deposit=int(input("Enter initial deposit amount: "))
    if initial_deposit < 0:
      return "initial deposit cannot be negative"

    #yeni hesap verilerini ekliyorum
    accounts[self.acc_no]={
        "name":name,
        "age":age,
        "gender":gender,
        "balance":initial_deposit
    }

    self.save_accounts(accounts)
    print(f"Account for {name} created successfully with account number: {self.acc_no}")

    return self.acc_no

    return f"Account created successfully.Account number: {acc_no}"


  def close_account(self,accounts):
    self.acc_no=input("Enter the account number to close: ")
    if self.acc_no in accounts:
      del accounts[self.acc_no]
      self.save_accounts(accounts)#hesap silindikten sonra kaydet


      return f"Account {self.acc_no} closed successfully."
    else:
      return f"Account number {self.acc_no} doesnt exist."

#sanırım dosyayı 'a' ekleme modunda açıcaz 'w' yazma modu değil yazma modu sonra geliyor
#çok sıkıntı olursa veri tabanı işlemlerini yap bunu boşver
#w mevcut içeriğin üzerine yazdırıcak
#a dosyanın sonuna ekliyicek bu yüzden a yı kullanmam lazım
#transac.json para yatırma para çekme işlemlerinde kendini yenilemiyor her işlemi kaydetmiyor neden bul
#sadece son işlemi kaydediyor yani yaptığının üstüne dosyayı yazıyor


In [17]:
#accounts={} bir kere açılıyor daha sonra kapatacaksın ki liste tekrar etmesin bunu da kontrol et geri dön düzelt
#transactions={} yukarıdaki gibi aynı şekilde

bank=Bank()
#bank.create_account(accounts)
bank.deposite_Amount(accounts,transactions)
#bank.create_account(accounts)
#bank.transfer_money(accounts,transactions)


                                  DEPOSİTE AMOUNT
    
Account Number: 11245648629649

        User Details {'name': 'Anıl', 'age': 958454, 'gender': 'mjdfngjdfn 7', 'balance': 65156165}
        
Deposite Amount: 50
